On order to be able to train with GPU:
Edit->Notebook Settings->Hardware Accelerator->Select GPU

Execute one cell after the other.

The Dataset has to be uploaded as a .zip File to your Google drive, containing all images and the csv file

The first Cell contains the Model Definition

Paste your Model here:


In [15]:
import torch
import torch.nn as nn

class DenseLayer(nn.Module):        #also called bottleneck Layer (Used for DenseNet-BC Variants)
    def __init__(self, in_channels, out_channels):
        super(DenseLayer, self).__init__()

        self.bn1 = nn.BatchNorm2d(in_channels)
        self.relu1 = nn.ReLU(inplace=True)
        self.conv1 = nn.Conv2d(in_channels, out_channels * 4, kernel_size = 1, stride = 1, padding = 0, bias = False)
        #1x1 kernel hat ko + k*(l-1) input channels, und erzeugt 4*32 outputs, welche zu 3x3 kernel gehen. Dies dient vor allem der Parameterreduzierung
        self.bn2 = nn.BatchNorm2d(out_channels * 4)
        self.relu2 = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels * 4, out_channels, kernel_size = 3, stride = 1, padding = 1, bias = False)
        
    def forward(self, x):
        out1 = self.conv1(self.relu1(self.bn1(x)))  #input wird zunächst von 1x1 kernel gefiltert
        out = self.conv2(self.relu2(self.bn2(out1))) #anschließend wird 3x3 kernel genutzt, welcher 32 output channels erzeugt
        return torch.cat([x, out], 1)

class TransitionBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(TransitionBlock, self).__init__()
                
        self.bn = nn.BatchNorm2d(in_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size = 1, stride = 1, padding = 0, bias = False)
        self.pool = nn.AvgPool2d(kernel_size=2,stride=2,padding=0)
        
    def forward(self, x):
        out = self.pool(self.conv(self.relu(self.bn(x)))) 
        return out
    
class DenseBlock(nn.Module):
    def __init__(self, layer_size, in_channels, growth_rate):   #growth_rate (k in paper) is equal to amount of output channels for each layer
        super(DenseBlock, self).__init__()
        
        self.block = []
        for i in range(layer_size):
            self.block.append(DenseLayer(in_channels+i*growth_rate,growth_rate))      #input = k0 + (l-1)*k, k0 = in_channels, l-1 = i (l = amount of layers before current layer)
            
        self.block = nn.Sequential(*self.block)
        
    def forward(self, x):
        out = self.block(x)
        return out
    
class DenseNet(nn.Module):
    def __init__(self, in_channels=2, layer_size=[6,6,7,8], growth_rate=12, additional_neurons = 8):  #in_channels describes picture channels (3 for rgb)  #layer_size must be list of 4 Elements
        super(DenseNet, self).__init__()
        
        self.out_channels = 2 * growth_rate #for k = 32 ->  64
        self.in_channels = 0    #used later
        
        #First Convolution & Pooling
        self.conv1 = nn.Conv2d(in_channels, self.out_channels, kernel_size=7, stride = 2, padding = 3, bias = False)
        self.batchnorm1 = nn.BatchNorm2d(self.out_channels)
        self.relu1 = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size = 3, stride = 2, padding = 1)
        
        #FirstDenseBlock
        self.in_channels = self.out_channels #2*growth_rate -> 64
        self.out_channels = growth_rate
        self.DenseBlock1 = DenseBlock(layer_size[0], self.in_channels, growth_rate) #6 Denselayer
        
        #First Transition Layer
        self.in_channels = self.in_channels + growth_rate * layer_size[0] #256
        self.out_channels = int(self.in_channels/2) #128
        self.TransitionLayer1 = TransitionBlock(self.in_channels, self.out_channels)
        
        #Second DenseBlock
        self.in_channels = self.out_channels    #128 in channels
        self.out_channels = growth_rate
        self.DenseBlock2 = DenseBlock(layer_size[1], self.in_channels, growth_rate) #12 Denselayer         
        
        #Second Transition Layer
        self.in_channels = self.in_channels + growth_rate * layer_size[1] #512
        self.out_channels = int(self.in_channels/2) #256
        self.TransitionLayer2 = TransitionBlock(self.in_channels, self.out_channels)
        
        #Third DenseBlock
        self.in_channels = self.out_channels    #256 in channels
        self.out_channels = growth_rate
        self.DenseBlock3 = DenseBlock(layer_size[2], self.in_channels, growth_rate) #24 Denselayer  
        
        #Third Transition Layer
        self.in_channels = self.in_channels + growth_rate * layer_size[2] #1024
        self.out_channels = int(self.in_channels/2) #512
        self.TransitionLayer3 = TransitionBlock(self.in_channels, self.out_channels)
        
        #Fourth DenseBlock
        self.in_channels = self.out_channels    #512 in channels
        self.out_channels = growth_rate
        self.DenseBlock4 = DenseBlock(layer_size[3], self.in_channels, growth_rate) #16 Denselayer
        
        #Global Average Pooling -> Compresses all channels of size 3x2 to size 1x1 (for input neurons)
        self.global_avg_pool = nn.AvgPool2d(kernel_size=(2,3),stride=4,padding=0)       #kernel size of 3x2 depends on input size of image!!!!
        
        self.in_channels = self.in_channels + growth_rate*layer_size[3] #1024
        self.batchnorm2 = nn.BatchNorm2d(self.in_channels)
        
        #fully connected layer
        self.in_channels = self.in_channels+additional_neurons
        self.fully_connected = nn.Linear(self.in_channels, 4)     #in_features = 1024 + additional neurons used as input, output = 4 (steering_left, steering_right, acceleration, brake)
        
        self.sigmoid = nn.Sigmoid()
        
        #initialization of all weights
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.constant_(m.bias, 0)
                
    def forward(self, x, a):   #x = Tensor of Image (96*85*in_channels -> specified in constructor of DenseNet), a = List of other Inputs
        x = self.maxpool(self.relu1(self.batchnorm1(self.conv1(x))))
        x = self.TransitionLayer1(self.DenseBlock1(x))       
        x = self.TransitionLayer2(self.DenseBlock2(x))
        x = self.TransitionLayer3(self.DenseBlock3(x))
        x = self.DenseBlock4(x)
        x = self.global_avg_pool(x)
        x = self.batchnorm2(x)
        x = torch.flatten(x, 1)
        if not torch.cuda.is_available():
          a = torch.Tensor(a)
        a = torch.squeeze(a, 0)
        x = torch.squeeze(x, 0)
        if(len(a.size()) > 1):  #checks if a is passed as batch or single value
            x = torch.cat((x,a),1)      #concatenate tensor x and a alsong dimension 1, since dimension zero is reserved by batch
        else:
            x = torch.cat((x,a),0)      #concatenate tensor x and a along dimension 0 (in evaluation mode)
        output = self.fully_connected(x)
        output = self.sigmoid(output)
        return output
    

def count_parameters(model):        #Function to count learnable parameters of model
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

The second cell mounts google Drive to Colab.
You don't have to change anything in the code, just execute and follow the steps described.

In [ ]:
#make sure to execute this code snipped before you start training

from google.colab import drive

#you will be asked to enter the authorization code of your google account - only valid for one session
#Go to this URL in a browser in order to obtain your key: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.activity.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fexperimentsandconfigs%20https%3a%2f%2fwww.googleapis.com%2fauth%2fphotos.native&response_type=code
drive.mount('/content/gdrive')

"""
gdrive can now be accessed under: 
content/gdrive/MyDrive/
"""

The third cell unzips the training data file stored on your google drive.
You have to provide the path of the zip file in the code.

In [ ]:
#google drive connection - Paste the path where you stored the zip on your drive here
#go to files (folder on the left side) -> gdrive ->... and navigate to the training data folder (zip file)
#copy the path of the folder and the csv file (by clicking on the three dots) and paste them here

!unzip "/content/gdrive/MyDrive/training_v2.zip"

#files will now be unzipped and stored locally on google colab -> increases speed drastically

In the fourth cell the DataLoader is prepared. Therefore you have to provide the csv path and the root directory in line 14&15 (on google colab). Copy the path by clicking on the three dots.


In [12]:
import torch
import os
import pandas as pd
from skimage import io
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")
print("all libraries are imported successfully...")

csv_path = "/content/training_v2/labels.csv"      #provide csv path (same name as zipped folder, but now locally, not on drive)
root_dir = "/content/training_v2"                 #provide root dir (same name as zipped folder, but now locally, not on drive)

# create a dataset class
# torch.utils.data.Dataset is an abstract clas representing a dataset
class SteeringCommands(Dataset): 
    """ Steering Commands dataset """

    def __init__(self, csv_file, root_dir, transform=None): 
        """
            Args:
                csv_file (string): Path to the csv file with annotations.
                root_dir (string): Directory with all the images.
                transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.label_file = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transforms = transform

    def __len__(self): 
        return len(self.label_file)

    def __getitem__(self, idx): 
        if torch.is_tensor(idx): 
            idx = idx.tolist()

        img_name = os.path.join(self.root_dir, self.label_file.iloc[idx,11])
        image = io.imread(img_name)
        image = image[:,:,0:2]      #remove blue color channel from image
        commands = self.label_file.iloc[idx, 8:11]
        commands = np.asarray(commands)
        if commands[0] == 1.0:           #append 0/1 depending on turning key value -> defines input for right key
            commands = np.concatenate(([1.0],commands))
        else:
            commands = np.concatenate(([0.0],commands))
        if commands[1] == -1.0:           #append 0/1 depending on turning key value -> defines input for left key
            commands =  np.concatenate(([1.0],commands))
        else:
            commands = np.concatenate(([0.0],commands))
        commands = np.delete(commands, 2)
        image = image.astype('float')
        input_data = self.label_file.iloc[idx, 0:8]
        input_data = np.asarray(input_data)
        commands = commands.astype('float').reshape(-1)
        input_data = input_data.astype('float').reshape(-1)


        if self.transforms: 
            image = self.transforms(image)
            
        commands = torch.from_numpy(commands)
        input_data = torch.from_numpy(input_data)
        sample = {'image': image, 'input_data' : input_data,'commands': commands}
        return sample

class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""

    def __call__(self, image):

        # swap color axis because
        # numpy image: H x W x C
        # torch image: C X H X W
        image = image.transpose((2, 0, 1))
        image = torch.from_numpy(image)
        return image

# instantiate conversion class
convert_data = ToTensor()       #initialize convert_data class which is used to transpose image in steeringcommands class

#create first dataset containing all images 
transformed_dataset = SteeringCommands(csv_file=csv_path, root_dir=root_dir, transform=convert_data)
#transformed dataset receives csf_file path and image directory path as input, as well as the transform operation defined in the ToTensor() class, which is called for each getitem call


 
    
#ONLY FOR DEBUGGING PURPOSES:

# number of subprocesses to use for data loading
num_workers = 0

# samples per batch to load
batch_size = 4

"""
dataloader = DataLoader(transformed_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)

for i_batch, sample_batched in enumerate(dataloader):
    #print(sample_batched['commands'])
    pass
"""


all libraries are imported successfully...


"\ndataloader = DataLoader(transformed_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)\n\nfor i_batch, sample_batched in enumerate(dataloader):\n    #print(sample_batched['commands'])\n    pass\n"

This Cell contains the training function and will run for some time.
The Hyperparameters in Line 8 to 13 need to be adapted.


To change the optimizer to SGD, go to line 71 and replace 
optimizer = optim.Adam(DenseNet.parameters(), lr=0.001, betas=(0.9, 0.999))
with
optimizer = optim.SGD(DenseNet.parameters(), lr=0.01, momentum=0.9).
Research Values for learning rate and momentum!

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

#----------- Parameters to be adapted! ------------
validation = False   #specify whether validation shall be done
batch_size = 32  #hyper parameter - amount of images in one batch - will be passed to the NN at once
nr_epochs = 100   #hyper parameter - determnies how many times the whole training set gets looped through the Neural Network
calculate_mean_and_std = False    #decide whether mean and std shall be calculated for whole dataset - takes some time on colab
load_pretrained_model = False     #decide whether you want to load parameters for the Model that were trained previously
path_to_pretrained_data = "/content/trained_desnenet.pth" #only relevant if "load_pretrained_model" set to true, path to parameters of model
#--------------------------------------------------

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")     #check if gpu is available for training

print("Creating Model and Loading Trainingdata")
DenseNet = DenseNet(in_channels=2, layer_size=[6,6,8,7], growth_rate=12, additional_neurons = 8).to(device=device)  #initialize DenseNet - Adapt Parameters!!!

if(load_pretrained_model == True):
  DenseNet.load_state_dict(torch.load(path_to_pretrained_data)['model_state_dict'])    #uncomment, if you want to continue to train a previously saved progress

DenseNet = DenseNet.float()     #convert parameters of network to float

if torch.cuda.is_available():
    print("Using GPU for training")
    DenseNet = DenseNet.cuda()  #load densenet to gpu if available

DenseNet.train()    #activate train mode for Densenet - Batchnorm activated & allows to pass batches


mean_r = 1.1384    #initialize mean value for red input pixel normalization
mean_g = 194.1198    #initialize mean value for green input pixel normalization
std_r = 16.9998     #initialize standard deviation value for red input pixel normalization
std_g = 108.7109     #initialize standard deviation value for green input pixel normalization

loader = DataLoader(transformed_dataset, batch_size=len(transformed_dataset), num_workers=0)    #create a dataset which contains all elements of dataLoading.dataset, in order to calculate mean and std     

if calculate_mean_and_std == True:
  print("Calculating Mean & Std for Green & Red Pixles")
  for i_batch, sample_batched in enumerate(loader):   #calculate meand and std of the whole training set but individually for each channel and save values in variables
      mean_r = sample_batched['image'][:,0,:,:].mean()
      std_r = sample_batched['image'][:,0,:,:].std()
      mean_g = sample_batched['image'][:,1,:,:].mean()
      std_g = sample_batched['image'][:,1,:,:].std()

print("Mean Value red: ", mean_r,", Mean Value green: ", mean_g)
print("Standard Deviation red: ", std_r, ", Standard Deviation green: ", std_g)  #!make sure to use these values during validation / test!

transform = transforms.Compose(     #create transform operation which substracts mean for each image tensor and divides by std and also contains normal transpose trainformation for image
    [convert_data,
     transforms.Normalize(mean=[mean_r, mean_g], std=[std_r, std_g])])

normalized_data = SteeringCommands(csv_file=csv_path, root_dir=root_dir, transform=transform)    #create new normalized training set

normalized_loader = DataLoader(normalized_data, batch_size=batch_size, shuffle=True, num_workers=0)    #load normalized training set - set hyperparameter batch_size

""" Only for debugging purposes
for i_batch, sample_batched in enumerate(normalized_loader):
    print(i_batch, sample_batched['image'].size(), sample_batched['input_data'].size(),
          sample_batched['commands'].size())
    print(sample_batched['image'][0])
    print("Tpye Image: ",type(sample_batched['image'][0]))
    print("Type Commands:, ", type(sample_batched['commands'][0]))
"""

criterion = nn.BCELoss()    #defines loss function - Loss function used here is binary cross entropy loss (CEL for sigmoid)
if torch.cuda.is_available():
    criterion = nn.BCELoss().cuda()
optimizer = optimizer = optim.SGD(DenseNet.parameters(), lr=0.01, momentum=0.9)#optim.Adam(DenseNet.parameters(), lr=0.001, betas=(0.9, 0.999)) #define optimizer - set hyper parameters lr and betas
if(load_pretrained_model == True):
  optimizer.load_state_dict(torch.load(path_to_pretrained_data)['optimizer_state_dict'])

if(True):
    print("Starting Training")
    
    for epoch in range(nr_epochs):      #loop which trainis the NN with the normalized dataset "nr_epochs" times. - set hyperparameter nr_epochs 
        
        running_loss = 0    #variable to calculate runnning loss (only for output in console)
        
        for i, data in enumerate(normalized_loader):        #loop which goes through whole normalized dataset once
            if torch.cuda.is_available():
                images = data['image'].cuda()                         #split up data dictionary in images, commands(steering, acceleration,brake) and inputs(speed, abs, gyroscope, steering)
                commands = data['commands'].cuda()
                inputs = data['input_data'].cuda()
            if not torch.cuda.is_available():           #transfer tensors to gpu if available 
                images = data['image']
                commands = data['commands']
                inputs = data['input_data']
            
            optimizer.zero_grad()       #resets all gradients to zero 
            
            outputs = DenseNet(images.float(), inputs.float())  #calculate output of one batch, input tensors have to consist of float numbers
            loss = criterion(outputs.float(), commands.float())     #calculate loss, therefore convert function inputs (output of NN and labels) to float
            loss.backward()     #calculate gradient for each parameter based on loss -> dloss/dx
            
            optimizer.step()    #adapts values of NN
            
            running_loss += loss.item()     #add loss of this batch to running loss in order to calculate mean loss later on
            if i % 50 == 49:    # print every 50 mini-batches
                print('[%d, %5d] loss: %.3f' %
                      (epoch + 1, i + 1, running_loss / 50))   #print running loss on screen
                running_loss = 0.0  #reset running loss
                
        print("Epoch ", epoch + 1, " finished")
        if((epoch+1) % 5 == 0):
          print("Intermediate result saved.")   
          PATH = './trained_densenetepoch'+str(int(epoch) + 1)+'.pth' 
          torch.save({          #save current state dict of model
            'epoch': epoch,
            'model_state_dict': DenseNet.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss,
            }, PATH)
    
    print('Finished Training')


normalized_loader_validation = DataLoader(normalized_data, batch_size=1, shuffle=True, num_workers=0)


if(validation == True):
    print("Starting Validation")
    
    running_loss = 0
    
    DenseNet.eval()
    
    with torch.no_grad():
        for i, data in enumerate(normalized_loader_validation):        
            images = data['image']                          #split up data dictionary in images, commands(steering, acceleration,brake) and inputs(speed, abs, gyroscope, steering)
            commands = data['commands']
            inputs = data['input_data']
            if torch.cuda.is_available():           #transfer tensors to gpu if available 
                commands = commands.cuda()
                inputs = inputs.cuda()
                images = images.cuda()
            outputs = DenseNet(images.float(), inputs.float())
            commands = torch.squeeze(commands, 0)   #remove unnessecary dimension form commands tensor size:[1,4] -> size:[4]
            loss = criterion(outputs.float(), commands.float())   
            running_loss += loss.item()                       
            if i % 100 == 99:    # print every 50 mini-batches
                print('loss: %.3f' %
                      (running_loss / 100))
                print("Sample Tensor Output:", outputs)
                print("Sample Desired Output:", commands)
                running_loss = 0.0  
            
    print("Finished Validation")


#make sure to download the file after training is finished
print("Parameters Saved. Please make sure to download the file")
PATH = './trained_densenetepoch'+str(int(epoch) + 1)+'.pth'
torch.save({          #save current state dict of model
            'epoch': epoch,
            'model_state_dict': DenseNet.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss,
            }, PATH)

Creating Model and Loading Trainingdata
Using GPU for training
Mean Value red:  1.1384 , Mean Value green:  194.1198
Standard Deviation red:  16.9998 , Standard Deviation green:  108.7109
Starting Training
[1,    50] loss: 0.344
[1,   100] loss: 0.307
[1,   150] loss: 0.293
